# TPT reporting methodology

## Report generation process

### Principles

TPT reports are generated at the shareclass level, using datas coming from differents sources. A primary processing is applied to store the data in database in a unified fashion.

Thus generating a report is done by getting data from database and processing it to ouput all informations required to fill the report.

At present, not all necessary data is in database yet and some divergence still remains between clients. As a result missing data is read from files and per-client processing is applied when necessary.

### Building blocks

The TPT_generator package is divided into functionnal modules.
- The **Fetcher** class is responsible for fetching the data in database and returning it stored in a pandas Dataframe. 
- The **Data_Bucket** class is responsible for holding the data necessary for the computation and feeding it to the generator while making sure that the necessary processing has been performed.
- Processing of the data is done by multiple modules:
    - The **Data_Processor** class is responsible for general computation such as the distribution of instruments values over the shareclasses, formatting and cleaning some datas or assigning value based on some conditions... in short all processing that is not done by one of the following modules.
    - The **Cash_flow** class is responsible for computing the future cash flow.
    - The **SCR_Module** class is responsible for computing Solvency Capital Requirement for each Risk submodules.
    
### Generation steps

The generation process can be divided into several steps:
1. **Initialisation of the generator**  
    - Input generation parameters
    - Initialisation of Data_Bucket object
    - create an empty dataframe to fill  


2. **Getting data from database (once for whole subfund)**  
    Through the Fetcher object, data is obtained from the database and stored in dataframes inside the Data_Bucket object.
    - date (config)
    - client (config)
    - shareclass_isin (config)
    - source_dir (config)
    - shareclass_infos (db)
    - shareclass_nav (db)
    - subfund_infos (db)
    - fund_infos (db)
    - instruments (db)
    - instruments_infos (db)
    - distribution_matrix (None - computed)
    - processing_data (None - computed)
    - scr (None - computed)


3. **Processing data (once for whole subfund)**  
    Processing the data is done at the subfund level to ensure coherence of the results and to avoid performing the same   computations multiple times. To store the data required to perform the processing as well as the results we build a multi-index dataframe where the level 1 index is the name of the instruments and the level 2 index is the isin of the shareclass. This allows to perform the computation in vectorized way for the whole subfund.
    
    - Compute_distribution_matrix
    - Clean_instruments_infos
    - process_instruments
    - compute_processing_data


4. **Computing SCR (per shareclass (at the moment))**  
    - Interest Rate up (compute_97)
    - Interest Rate down (compute 98)
    - Market equity type1 (compute_99)
    - Market equity type2 (compute_100)
    - Market spread (compute_102)
    - Currency up (compute_105a)
    - Currency down (compute_105b)


5. **Filling the report**


6. **writing to file**

    - output_excel

###  Imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
from pathlib import Path
from TPT_generator_python import Data_Bucket, TPT_Fetcher, TPT_Generator

###  Initialisation

- date
- client
- isin of the shareclass
- source directory (containing missing data and templates. should be removed in the future.)
- output directory (for the generated report)
- symmetric adjustment

In [2]:
DATE = "2020-12-31"
CLIENT = "BIL"
ISIN = "LU1689730718"
SOURCE_DIR = Path("../data")
OUTPUT_DIR = './production/example'

In [3]:
# Initialise a TPT_Generator object:
# Define generating constants
# Initialise a Data_Bucket object to hold generating data
# Initialise an empty Dataframe of correct dimensions to hold the report's data.
generator = TPT_Generator(DATE,
                          CLIENT,
                          ISIN,
                          SOURCE_DIR,
                          OUTPUT_DIR)

print(generator)
generator.data_bucket.fetch()
print(generator.data_bucket)


    Reporting date:          2020-12-31
    Client:                  BIL
    Shareclass isin:         LU1689730718 
        

    fund infos:              (1, 10)
CODE:                                            549300IVXKQHV6O7PY61
NAME:                                            BIL Invest
COUNTRY:                                         LU
...

    subfund infos:           (1, 9)
CODE:                                            5493003HFW1KPBNTNU32
NAME:                                            BIL Invest Bonds EUR Corporate Investment Grade
CURRENCY:                                        EUR
...

    shareclass infos:        (3, 7)
ISIN:                                            LU1689730718
NAME:                                            BIL Invest Bonds EUR Corporate Investment Grade - I EUR CAP
CURRENCY:                                        EUR
...

ISIN:                                            LU1689730809
NAME:                                            BIL Invest B

###  Portfolio characteristics and valuation [1-11]

These columns have the same value for all rows of the report.
- Columns **1 and 11** are filled with data originating from config.
- Columns **2 to 8** are filled with data originating from database.
- Column **9** is the sum of the values in portfolio currency of cash account (CIC code XT72) over the total net asset of the shareclass in portfolio currency.
- Column **10** 

None


In [5]:
"', '".join(["isin"])

'isin'